In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
import torch.nn.init as init

from hyperopt import pyll, hp, fmin, tpe
from hyperopt.pyll import scope

use_cuda = torch.cuda.is_available()

In [2]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
            
            
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs


MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]


def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))


Reading lines...
Read 135842 sentence pairs
Trimmed to 10853 sentence pairs
Counting words...
Counted words:
fra 4489
eng 2925
[u'je suis satisfait de sa performance .', u'i m pleased with his performance .']


In [3]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        #self.gru = nn.GRU(hidden_size, hidden_size)
        
        
        #self.Wxh = nn.Parameter(torch.zeros(1,1,256)).cuda()
        
        #self.Wxh = nn.Linear(256,256)
        #self.Whh = nn.Linear(256,256)
        
        #self.Why = nn.Linear(256,256)
        
        self.rnn = nn.RNN(hidden_size,hidden_size,1)
        
#         self.Whh =nn.Parameter(torch.zeros(1,1,256)).cuda()
#         self.bh = nn.Parameter(torch.zeros(1,1,256)).cuda()
#         self.Why = nn.Parameter(torch.zeros(1,1,256)).cuda()
#         self.by = nn.Parameter(torch.zeros(1,1,256)).cuda()
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax()
        
        self.params = nn.ParameterList([nn.Parameter(torch.randn(10, 10)) for i in range(10)])

       

    def forward(self, input, hidden):
        
        embedded = self.embedding(input).view(1, 1, -1)
        
        output = embedded
        
        #output = output[0][0].view(1,256)
        hidden = hidden.view(1,1,-1)
        
#         balh = self.Wxh(output[0][0].view(1,256))
        
#         hidden = self.tanh(self.Wxh(output[0][0].view(1,256))+self.Whh(hidden)).view(1,256)
        
#         output = self.softmax((self.Why(hidden))).view(1,1,256)

        #print("Embedding size: ",output.size())
        #print("Hidden size: ",hidden.size())
        
        output,hidden = self.rnn(output,hidden)
        
        #output = output.view(-1,256)
        
        
        #output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        result = Variable(torch.zeros(1, self.hidden_size))
        if use_cuda:
            return result.cuda(1)
        else:
            return result

In [4]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        #self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax()
        
        self.rnn = nn.RNN(hidden_size,hidden_size,1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        
        output, hidden = self.rnn(output, hidden)
        #output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            return result.cuda(1)
        else:
            return result


In [5]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def variableFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    result = Variable(torch.LongTensor(indexes).view(-1, 1))
    if use_cuda:
        return result.cuda(1)
    else:
        return result


def variablesFromPair(pair):
    input_variable = variableFromSentence(input_lang, pair[0])
    target_variable = variableFromSentence(output_lang, pair[1])
    return (input_variable, target_variable)

In [6]:
teacher_forcing_ratio = 0.5


def train(input_variable, target_variable, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_variable.size()[0]
    target_length = target_variable.size()[0]

    encoder_outputs = Variable(torch.zeros(max_length, encoder.hidden_size))
    encoder_outputs = encoder_outputs.cuda(1) if use_cuda else encoder_outputs

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_variable[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0]

    decoder_input = Variable(torch.LongTensor([[SOS_token]]))
    decoder_input = decoder_input.cuda(1) if use_cuda else decoder_input

    decoder_hidden = encoder_hidden.view(1,1,-1)

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_variable[di])
            decoder_input = target_variable[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            ni = topi[0][0]

            decoder_input = Variable(torch.LongTensor([[ni]]))
            decoder_input = decoder_input.cuda(1) if use_cuda else decoder_input

            loss += criterion(decoder_output, target_variable[di])
            if ni == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.data[0] / target_length

In [7]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [19]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=1000, learning_rate=0.01,optimizer='SGD'):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every
    
    if optimizer == 'Adam':
        encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
        decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    else:
        encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
        decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

    training_pairs = [variablesFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    loss_initial = 0
    for iter in range(1, n_iters + 1):
       
        training_pair = training_pairs[iter - 1]
        input_variable = training_pair[0]
        target_variable = training_pair[1]
        
        loss = train(input_variable, target_variable, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        
        
            
        print_loss_total += loss
        

        
        plot_loss_total += loss

        if iter % print_every == 0:
            
            if iter == 1000:
                loss_initial = print_loss_total
            
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))
                                         
    return print_loss_avg/loss_initial

In [14]:


hidden_size = 300

encoder1 = EncoderRNN(input_lang.n_words, hidden_size)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words)

if use_cuda:
    encoder1 = encoder1.cuda(1)
    decoder1 = decoder1.cuda(1)

for name, p in encoder1.named_parameters():
    if len(p.size()) >= 2:
        init.uniform(p,-0.08,0.08)

trainIters(encoder1,decoder1, 2000, print_every=1000)

0m 18s (- 0m 18s) (1000 50%) 3.4969
0m 37s (- 0m 0s) (2000 100%) 3.1317


1116.6158522903122

In [20]:
def main(params):

    hidden_size = params['hidden_size']
    learning_rate = params['learning_rate']
    weights_init = params['weights_init']
    optimizer = params['optimizer']
    
    encoder1 = EncoderRNN(input_lang.n_words, hidden_size)
    decoder1 = DecoderRNN(hidden_size, output_lang.n_words)

    if use_cuda:
        encoder1 = encoder1.cuda(1)
        decoder1 = decoder1.cuda(1)

    for name, p in encoder1.named_parameters():
        if len(p.size()) >= 2:
            init.uniform(p,-weights_init,weights_init)

    return trainIters(encoder1,decoder1, 75000, print_every=1000,learning_rate=learning_rate,optimizer=optimizer)

In [ ]:
space = {'weights_init': hp.uniform('weights_init', -0.1, 0.1) ,
'hidden_size':scope.int(hp.quniform('hidden_size',500,4000,50)),
'learning_rate':hp.uniform('learning_rate',0.005, 0.05),
'optimizer':hp.choice('optimizer_name',('Adam', 'SGD'))
}

best = fmin(main, space, algo=tpe.suggest, max_evals=5)
print("best parameters", best)


In [208]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    input_variable = variableFromSentence(input_lang, sentence)
    input_length = input_variable.size()[0]
    encoder_hidden = encoder.initHidden()

    encoder_outputs = Variable(torch.zeros(max_length, encoder.hidden_size))
    encoder_outputs = encoder_outputs.cuda(1) if use_cuda else encoder_outputs

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_variable[ei],
                                                 encoder_hidden)
        encoder_outputs[ei] = encoder_outputs[ei] + encoder_output[0]

    decoder_input = Variable(torch.LongTensor([[SOS_token]]))  # SOS
    decoder_input = decoder_input.cuda(1) if use_cuda else decoder_input

    decoder_hidden = encoder_hidden.view(1,1,-1)

    decoded_words = []
    decoder_attentions = torch.zeros(max_length, max_length)

    for di in range(max_length):
        decoder_output, decoder_hidden = decoder(
            decoder_input, decoder_hidden)
        topv, topi = decoder_output.data.topk(1)
        ni = topi[0][0]
        if ni == EOS_token:
            decoded_words.append('<EOS>')
            break
        else:
            decoded_words.append(output_lang.index2word[ni])

        decoder_input = Variable(torch.LongTensor([[ni]]))
        decoder_input = decoder_input.cuda(1) if use_cuda else decoder_input

    return decoded_words

In [171]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [172]:
evaluateRandomly(encoder1, decoder1)

> je suis une bonne personne .
= i m a good person .
< i m not of . . <EOS>

> il lit .
= he is reading .
< he s a . <EOS>

> je suis fatigue de danser .
= i m tired of dancing .
< i m not of . . <EOS>

> nous sommes des nouveaux venus .
= we re newcomers .
< we re all here . <EOS>

> vous etes tres raffinee .
= you re very sophisticated .
< you are very understanding . <EOS>

> je suis desolee mon pere est dehors .
= i m sorry my father is out .
< i m very proud of our . <EOS>

> tu es timide .
= you re shy .
< you re sick . <EOS>

> elle est une championne de football .
= she s a soccer champion .
< she s a at of . <EOS>

> je creve d envie de la revoir .
= i am dying to see her again .
< i m very proud of our . <EOS>

> je me sens assez confiant .
= i m feeling pretty confident .
< i m not of . . <EOS>



In [197]:
list(encoder1.named_parameters())

[('embedding.weight', Parameter containing:
  -1.4506e+00 -1.1463e+00 -1.1788e-01  ...  -1.4200e+00 -7.5380e-01 -8.4167e-01
   9.7898e-01  6.0189e-01  1.0324e+00  ...   6.2386e-01  1.1707e+00  6.2751e-01
   3.3029e-02 -2.4832e-01  1.0357e-01  ...  -1.0030e+00 -1.4789e+00 -4.5356e-01
                  ...                   ⋱                   ...                
  -7.3199e-01 -1.4512e+00  2.0953e-01  ...   3.9212e-01  6.5632e-01  6.7086e-01
  -7.1268e-01  1.0285e+00  1.2198e+00  ...   9.6443e-02  4.6183e-01 -9.8866e-01
  -1.5849e-01  9.2358e-01 -2.7277e+00  ...   1.3562e+00 -1.6801e+00  9.9849e-01
  [torch.cuda.FloatTensor of size 4489x256 (GPU 0)]),
 ('Wxh.weight', Parameter containing:
   3.9429e-02 -5.3712e-02 -4.0545e-02  ...  -2.1362e-02 -3.2580e-02  1.9622e-04
   5.2920e-02  2.2854e-02  4.1614e-02  ...   5.9415e-02 -5.4396e-02 -5.1029e-02
   5.8379e-02 -2.6421e-02 -4.6973e-02  ...   4.5297e-02  4.7184e-02 -4.4916e-02
                  ...                   ⋱                   ... 